In [1]:
import numpy as np
from scipy.stats import norm, bernoulli
from scipy.sparse import csc_matrix
import datetime as dt

In [2]:
x = dt.date.today()
y = dt.date(2023, 1, 20)
z = y-x
z

datetime.timedelta(days=160)

In [3]:
# Leg A
r = 0.05 #Risk Free Rate
s = 36.15 #Uderlying
k = 40 #Strik
T = 168/365 #Time
sigma = 0.7483 #Vol

In [4]:
def bsm(r,s,k,T,sigma,type = 'p'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  d2 = d1 - sigma*np.sqrt(T)
  try:
    if type == 'c':
      price = s*norm.cdf(d1, 0, 1) - k*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif type == 'p':
      price = k*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - s*norm.cdf(-d1, 0, 1)
    return price
  except:
    print('Please Confirm all Parameteres')

def delta(r,s,k,T,sigma,type = 'p'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  try:
    if type == 'c':
      delta = norm.cdf(d1, 0, 1)
    elif type == 'p':
      delta = -norm.cdf(-d1, 0, 1)
    return delta
  except:
    print('Please Confirm all Parameteres')

def gamma(r,s,k,T,sigma,type = 'c'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  try:
      gamma = norm.pdf(d1, 0, 1)/(s*sigma*np.sqrt(T))
      return gamma
  except:
    print('Please Confirm all Parameteres')

def vega(r,s,k,T,sigma,type = 'c'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  d2 = d1 - sigma*np.sqrt(T)
  try:
    vega = s*norm.pdf(d1, 0, 1)*np.sqrt(T)
    return vega
  except:
    print('Please Confirm all Parameteres')

def theta(r,s,k,T,sigma,type = 'c'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  d2 = d1 - sigma*np.sqrt(T)
  try:
    if type == 'c':
      theta = -s*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*k*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif type == 'p':
      theta = -s*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*k*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
    return theta
  except:
    print('Please Confirm all Parameteres')

def rho(r,s,k,T,sigma,type = 'c'):
  d1 = (np.log(s/k) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
  d2 = d1 - sigma*np.sqrt(T)
  try:
    if type == 'c':
      rho = k*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif type == 'p':
      rho = -k*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
    return rho
  except:
    print('Please Confirm all Parameteres')

In [5]:
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt

In [6]:
symbol = 'SPY'
tk = yf.Ticker(symbol)
expiry = '2023-01-20'

In [9]:
close = tk.info['regularMarketPrice']
close

427.1

In [10]:
 # Get options exp
options = pd.DataFrame()
opt = tk.option_chain(expiry.strip())
opt = pd.DataFrame().append(opt.calls).append(opt.puts)
opt['expirationDate'] = expiry
options = options.append(opt, ignore_index=True)

# Add 1 day to get the correct expiration date
options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365

# Boolean column if the option is a CALL
options['CALL'] = options['contractSymbol'].str[4:].apply(
    lambda x: "C" in x)

options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)

# Drop unnecessary and meaningless columns
options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])

options['CALL'] = options['CALL'].replace(True, 'c')
options['CALL'] = options['CALL'].replace(False, 'p')

C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3192316074.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3192316074.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(opt, ignore_index=True)


In [11]:
call_df = options.loc[lambda options: options['CALL'] == 'c']
put_df = options.loc[lambda options: options['CALL'] == 'p']

In [12]:
call_df['BSM Value'] = bsm(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df['Delta'] = delta(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df['Gamma'] = gamma(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df['Vega'] = vega(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df['Theta'] = theta(r, call_df['lastPrice'], call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df['Rho'] = rho(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
call_df

C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3492779638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_df['BSM Value'] = bsm(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3492779638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  call_df['Delta'] = delta(r, close, call_df['strike'], call_df['dte'], call_df['impliedVolatility'], type = 'c')
C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3492779638.py

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,BSM Value,Delta,Gamma,Vega,Theta,Rho
0,SPY230120C00165000,165.0,244.74,261.67,262.49,2.0,18.0,0.620121,True,2023-01-21,0.438356,c,265.992643,0.994990,0.000083,4.096307,-28.133707,69.684383
1,SPY230120C00170000,170.0,205.16,241.57,243.15,1.0,0.0,0.000010,True,2023-01-21,0.438356,c,260.785491,1.000000,0.000000,0.000000,-8.315725,72.904990
2,SPY230120C00175000,175.0,208.76,231.37,233.33,1.0,10.0,0.000010,True,2023-01-21,0.438356,c,255.893888,1.000000,0.000000,0.000000,-8.560306,75.049255
3,SPY230120C00180000,180.0,207.01,226.43,228.38,2.0,4.0,0.000010,True,2023-01-21,0.438356,c,251.002285,1.000000,0.000000,0.000000,-8.804886,77.193519
4,SPY230120C00185000,185.0,193.49,212.97,214.90,6.0,9.0,0.000010,True,2023-01-21,0.438356,c,246.110681,1.000000,0.000000,0.000000,-9.049466,79.337784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,SPY230120C00695000,695.0,0.01,0.00,0.01,1.0,115.0,0.222664,False,2023-01-21,0.438356,c,0.017395,0.001034,0.000055,0.981684,-0.000000,0.185938
130,SPY230120C00700000,700.0,0.01,0.00,0.01,2.0,2237.0,0.224617,False,2023-01-21,0.438356,c,0.016314,0.000966,0.000051,0.922472,-0.000000,0.173726
131,SPY230120C00705000,705.0,0.01,0.00,0.05,2.0,89.0,0.259773,False,2023-01-21,0.438356,c,0.074091,0.003461,0.000142,2.942471,-0.000000,0.615555
132,SPY230120C00710000,710.0,0.01,0.00,0.02,120.0,911.0,0.243172,False,2023-01-21,0.438356,c,0.031059,0.001640,0.000077,1.496921,-0.000000,0.293457


In [13]:
put_df['BSM Value'] = bsm(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
put_df['Delta'] = delta(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
put_df['Gamma'] = gamma(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'c')
put_df['Vega'] = vega(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'c')
put_df['Theta'] = theta(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
put_df['Rho'] = rho(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
put_df

C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3324677664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  put_df['BSM Value'] = bsm(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3324677664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  put_df['Delta'] = delta(r, close, put_df['strike'], put_df['dte'], put_df['impliedVolatility'], type = 'p')
C:\Users\ritwi\AppData\Local\Temp\ipykernel_2036\3324677664.py:3: Sett

,contractSymbol,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,BSM Value,Delta,Gamma,Vega,Theta,Rho
134,SPY230120P00165000,165.0,0.21,0.20,0.21,118.0,14333.0,0.574711,False,2023-01-21,0.438356,p,0.165023,-0.003004,0.000056,2.590317,-1.625628,-0.634761
135,SPY230120P00170000,170.0,0.24,0.23,0.25,126.0,5934.0,0.567875,False,2023-01-21,0.438356,p,0.192765,-0.003504,0.000066,2.975093,-1.842598,-0.740537
136,SPY230120P00175000,175.0,0.25,0.26,0.28,90.0,6406.0,0.559086,False,2023-01-21,0.438356,p,0.217232,-0.003969,0.000074,3.327019,-2.026041,-0.838361
137,SPY230120P00180000,180.0,0.28,0.29,0.31,20.0,5212.0,0.549809,False,2023-01-21,0.438356,p,0.241780,-0.004450,0.000084,3.685091,-2.203901,-0.939095
138,SPY230120P00185000,185.0,0.32,0.32,0.34,70.0,4182.0,0.540044,False,2023-01-21,0.438356,p,0.265896,-0.004940,0.000094,4.044802,-2.372768,-1.041348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,SPY230120P00680000,680.0,229.00,254.80,259.06,NaN,0.0,0.494207,True,2023-01-21,0.438356,p,245.158158,-0.883126,0.001405,55.521097,-0.180428,-272.807154
252,SPY230120P00690000,690.0,223.50,243.30,247.07,NaN,1.0,0.000010,True,2023-01-21,0.438356,p,247.941243,-1.000000,0.000000,0.000000,33.752062,-295.908490
253,SPY230120P00695000,695.0,295.76,316.12,319.21,19.0,0.0,1.023305,True,2023-01-21,0.438356,p,304.532429,-0.635907,0.001298,106.200218,-95.151261,-252.549363
254,SPY230120P00700000,700.0,323.63,314.61,315.42,1.0,0.0,0.956330,True,2023-01-21,0.438356,p,301.690662,-0.666073,0.001345,102.889554,-82.924800,-256.951496


In [ ]:
def getInput():
    '''
    gets the user input on symbol and expiry
    '''
    symbol = input("Enter the Symbol: " )
    
    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options   
    print("Expiry dates:")        
    for i in exps:
        print(f"{i}", end=", ")
  
    
    expiry = input("\nEnter the Expiry data [format: YYYY-MM-DD]: " )
    return tk, expiry, symbol

In [ ]:
getInput()

In [ ]:
ty = [tk.options]

In [ ]:
print('Available Expiry Dates', ty)